# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [18]:
# Use as many python cells as you wish to write your code
# Define the following libraries
import pandas as pd
import numpy as np

In [19]:
# Define the merge_all_data function
def merge_all_data(user_health_data_path, supplement_usage_path, experiments_path, user_profiles_path):
    user_health_data = pd.read_csv(user_health_data_path)
    supplement_usage = pd.read_csv(supplement_usage_path)
    experiments = pd.read_csv(experiments_path)
    user_profiles = pd.read_csv(user_profiles_path)

    user_health_data = user_health_data.dropna(subset=['user_id', 'date'])
    user_health_data['user_id'] = user_health_data['user_id'].str.lower().str.strip()
    user_health_data['date'] = pd.to_datetime(user_health_data['date'])
    user_health_data['sleep_hours'] = user_health_data['sleep_hours'].str.strip()
    user_health_data['sleep_hours'] = user_health_data['sleep_hours'].str.replace('h', '', case=False)

    supplement_usage = supplement_usage.dropna(subset=['user_id', 'date'])
    supplement_usage['user_id'] = supplement_usage['user_id'].str.lower().str.strip()
    supplement_usage['date'] = pd.to_datetime(supplement_usage['date'])
    supplement_usage['supplement_name'] = supplement_usage['supplement_name'].str.strip()
    supplement_usage['dosage_unit'] = supplement_usage['dosage_unit'].str.lower().str.strip()
    supplement_usage['experiment_id'] = supplement_usage['experiment_id'].str.lower().str.strip()
    supplement_usage['dosage_grams'] = supplement_usage.loc[supplement_usage['dosage_unit'] == 'mg', 'dosage'] / 1000

    experiments['experiment_id'] = experiments['experiment_id'].str.lower().str.strip()
    experiments['name'] = experiments['name'].str.strip()

    user_profiles = user_profiles.dropna(subset=['user_id', 'email'])
    user_profiles['user_id'] = user_profiles['user_id'].str.lower().str.strip()
    user_profiles['email'] = user_profiles['email'].str.lower().str.strip()

    def age_to_group(age):
        if pd.isna(age):
            return "Unknown"
        elif age < 18:
            return "Under 18"
        elif age <= 25:
            return "18-25"
        elif age <= 35:
            return "26-35"
        elif age <= 45:
            return "36-45"
        elif age <= 55:
            return "46-55"
        elif age <= 65:
            return "56-65"
        else:
            return "Over 65"

    user_profiles['user_age_group'] = user_profiles['age'].apply(age_to_group)

    user_data = pd.merge(user_profiles[['user_id', 'email', 'user_age_group']], user_health_data, on=['user_id'], how='left')
    user_supplements = pd.merge(user_data, supplement_usage, on=['user_id', 'date'], how='left')
    user_experiments = pd.merge(user_supplements, experiments, on=['experiment_id'], how='left')
    user_experiments = user_experiments.rename(columns={'name': 'experiment_name'})

    df = user_experiments[['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']]
    
    df['supplement_name'] = df['supplement_name'].fillna('No intake')
    df.loc[df['supplement_name'] == 'No intake', 'dosage_grams'] = np.nan
    df.loc[df['dosage_grams'] == np.nan, 'is_placebo'] = np.nan
    
    df['user_id'] = df['user_id'].astype('string')
    df['email'] = df['email'].astype('string')
    df['user_age_group'] = df['user_age_group'].astype('category')
    df['supplement_name'] = df['supplement_name'].astype('category')
    df['dosage_grams'] = df['dosage_grams'].astype(float)
    df['average_heart_rate'] = df['average_heart_rate'].astype(float)
    df['average_glucose'] = df['average_glucose'].astype(float)
    df['sleep_hours'] = df['sleep_hours'].astype(float)

    df = df.dropna(subset=['user_id', 'date', 'email'])
    
    return df

In [20]:
# Call the function
merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')

user_id  ... activity_level
0     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
1     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              3
2     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
3     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
4     c6ae338a-9f95-481c-a88d-24a58bc8fc71  ...              1
...                                    ...  ...            ...
2716  9c083dc3-b789-4cee-87fb-9887ff4b4e29  ...              3
2717  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              1
2718  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4
2719  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              3
2720  2f2aa322-d2b2-4c88-aeb6-e49e768521f3  ...              4

[2721 rows x 12 columns]